In [4]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

In [5]:
df = pd.read_csv('all_reviews.csv')

In [6]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
9684,Port Orleans Bacchus Blonde,4.6,"New Orleans, Louisiana",Port Orleans Brewing Company,"Bacchus Blonde has a thick, soapy white head, ...",ronaldtheriot
9685,Harcos (Hungary),2.7,"Neckarsulm, Baden-Württemberg",Lidl Europe,I Tasted as B Lidl Warrior IPA /B /I Very good...,JorisPPattyn
9686,Deviant & Dandy Lucid Dream,3.4,"Homerton, Greater London",Deviant & Dandy Brewery,Can from the Plumsteadshire online shop. Hazy ...,planky84
9687,League Night Lager,2.9,"Baileys Harbor, Wisconsin",Door County Brewing Company,Light yellow pour with soda pop white head. Gr...,pkbites


In [ ]:
geolocator = Nominatim(user_agent="None")

In [9]:
geolocator.geocode("Bungay, Suffolk").latitude

52.4562248

In [14]:
locations_df = pd.DataFrame(df['brewery_location'].unique(),columns=["locations"])

In [16]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [17]:
locations_df["latitude"] = locations_df.locations.apply(lambda x: tryconvert(x))

In [18]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [19]:
locations_df["longitude"] = locations_df.locations.apply(lambda x: tryconvertlong(x))